# Load dataset

In [1]:
#Acknowlegement: Some codes used to initialise BERT are from BERT tutorial(https://colab.research.google.com/drive/1Y4o3jh3ZH70tl6mCd76vz_IxX23biCPP)

In [2]:
# Import necessary libraries
import re
import string
import os 
from collections import defaultdict
from collections import Counter
import time
import numpy as np
import torch
import datetime
import copy
from transformers import get_linear_schedule_with_warmup
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from covid_tools import *
from BERTs import *

I1218 14:20:00.545545 139713408845632 driver.py:124] Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
I1218 14:20:00.561142 139713408845632 driver.py:124] Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt


In [3]:
# check GPU situation
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Dec 18 14:20:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:DA:00.0 Off |                  N/A |
| 41%   29C    P8     9W / 280W |      3MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# load orginal data(COVID-19 TREC-IS 2020 task) from json file.
cv19_dc = pd.read_json('./Pythonbooks/Data/COVID/cv19_dc_test.json',lines=True,orient='records')
cv19_ws = pd.read_json('./Pythonbooks/Data/COVID/cv19_washington_state_test.json',lines=True,orient='records')
cv19_ny = pd.read_json('./Pythonbooks/Data/COVID/cv19_nyc_test.json',lines=True,orient='records')
cv19_a_run_info_labeled = pd.read_csv('cache/cv19_TREC_2020_all_labeled_A.csv',
                           converters={"priority": lambda x: x.strip("[]").replace("'", "").split(", "),
                                       "categories": lambda x: x.strip("[]").replace("'", "").split(", ")})
cv19_dc_labeled = pd.merge(cv19_dc,cv19_a_run_info_labeled,on=['id'])
cv19_ws_labeled = pd.merge(cv19_ws,cv19_a_run_info_labeled,on=['id'])
cv19_ny_labeled = pd.merge(cv19_ny,cv19_a_run_info_labeled,on=['id'])
temp = [cv19_dc_labeled,cv19_ny_labeled,cv19_ws_labeled ]
cv19_a_run_info_labeled =  pd.concat(temp)
cv19_a_run_info_labeled = cv19_a_run_info_labeled.reset_index()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [5]:
cv19_b_run_info_labeled = pd.read_csv('cache/cv19_TREC_2020_all_labeled_B.csv',
                           converters={"priority": lambda x: x.strip("[]").replace("'", "").split(", "),
                                       "categories": lambda x: x.strip("[]").replace("'", "").split(", ")})
cv19_Houston_2020 = pd.read_json('./Pythonbooks/Data/COVID/cv_Houston_2020.json',lines=True,orient='records')
cv19_Atlanta_2020 = pd.read_json('./Pythonbooks/Data/COVID/cv_Atlanta_2020.json',lines=True,orient='records')
cv19_Melbourne2020 = pd.read_json('./Pythonbooks/Data/COVID/cv_Melbourne_2020.json',lines=True,orient='records')
cv19_NYC_2020 = pd.read_json('./Pythonbooks/Data/COVID/cv_NYC_2020.json',lines=True,orient='records')
cv19_acksonville_2020 = pd.read_json('./Pythonbooks/Data/COVID/cv_Jacksonville_2020.json',lines=True,orient='records')

# Merge original dataset with lablled dataset based on id of tweets
cv19_Houston_2020_labeled = pd.merge(cv19_Houston_2020, cv19_b_run_info_labeled,on=['id'])
cv19_Atlanta_2020_labeled = pd.merge(cv19_Atlanta_2020, cv19_b_run_info_labeled,on=['id'])
cv19_Melbourne2020_labeled = pd.merge(cv19_Melbourne2020, cv19_b_run_info_labeled,on=['id'])
cv19_NYC_2020_labeled = pd.merge(cv19_NYC_2020, cv19_b_run_info_labeled, on=['id'])
cv19_acksonville_2020_labeled = pd.merge(cv19_acksonville_2020,cv19_b_run_info_labeled, on=['id'])
temp = [cv19_Houston_2020_labeled,cv19_Atlanta_2020_labeled,cv19_Melbourne2020_labeled,cv19_NYC_2020_labeled, cv19_acksonville_2020_labeled ]
cv19_b_run_info_labeled =  pd.concat(temp)
cv19_b_run_info_labeled = cv19_b_run_info_labeled.reset_index()
cv19_b_run_info_labeled = cv19_b_run_info_labeled.rename(columns={'full_text_x':'full_text'})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [6]:
len(cv19_b_run_info_labeled )

4505

# Pre-process the data 

In [7]:
categories = ['GoodsServices','InformationWanted','Volunteer','EmergingThreats','NewSubEvent','ServiceAvailable','Advice',]

# # Extract training set
# featureList = ['full_text'] #,'favorited'
# dataset_inputs = extract_features(Tweets,featureList)

# Extract labels
labelList = ['categories']
a_labels = Process_labels.extract_features(cv19_a_run_info_labeled,labelList)['categories']
b_labels = Process_labels.extract_features(cv19_b_run_info_labeled,labelList)['categories']
# Transfer a list of text labels into a list of number labels, like [0,1,0,0,0,0,0]
a_labels = Process_labels.extractLabels(cv19_a_run_info_labeled,a_labels)
b_labels = Process_labels.extractLabels(cv19_b_run_info_labeled,b_labels)

In [8]:
len(cv19_b_run_info_labeled)

4505

#### Build label for each catergory

In [9]:
def build_single_label(labels):
    labels_single = {}
    for i in range(7):
      print('Currrent is processing on categorie %s'%(categories[i]))
      print()
      # if i == 3:
      #   continue
      labels_one_category = []
      for j in range(len(labels)):
        labels_one_category.append(labels[j][i])
      labels_single[i] = labels_one_category
    return labels_single

In [10]:
categories = ['GoodsServices','InformationWanted','Volunteer','EmergingThreats','NewSubEvent','ServiceAvailable','Advice']
# Sperate labels for each category and store them in one dict
# A dict Contains labels for all categories e.g. trainL_single[category]
labels_single_a = build_single_label(a_labels)
labels_single_b = build_single_label(b_labels)

Currrent is processing on categorie GoodsServices

Currrent is processing on categorie InformationWanted

Currrent is processing on categorie Volunteer

Currrent is processing on categorie EmergingThreats

Currrent is processing on categorie NewSubEvent

Currrent is processing on categorie ServiceAvailable

Currrent is processing on categorie Advice

Currrent is processing on categorie GoodsServices

Currrent is processing on categorie InformationWanted

Currrent is processing on categorie Volunteer

Currrent is processing on categorie EmergingThreats

Currrent is processing on categorie NewSubEvent

Currrent is processing on categorie ServiceAvailable

Currrent is processing on categorie Advice



## Tokenization: Bert tokenizer

In [11]:
import pandas as pd
dataset_inputs_A_run = pd.read_csv('priority/a_run/dataset_all.csv')
# B_run_temp = pd.read_csv('priority/b_run/dataset_all.csv')

In [12]:
# dataset_inputs_A_run = pd.read_csv('cache/cv19_TREC_2020_all_labeled_A.csv',
#                            converters={"priority": lambda x: x.strip("[]").replace("'", "").split(", "),
#                                        "categories": lambda x: x.strip("[]").replace("'", "").split(", ")})

# dataset_inputs_B_run = pd.read_csv('cache/cv19_TREC_2020_all_labeled_B.csv',
#                            converters={"priority": lambda x: x.strip("[]").replace("'", "").split(", "),
#                                        "categories": lambda x: x.strip("[]").replace("'", "").split(", ")})

# B_run_temp = pd.read_csv('priority/b_run/dataset_all.csv')
# dataset_inputs_B_run = pd.merge(cv19_b_run_info_labeled,B_run_temp,on=['id'])
# dataset_inputs_B_run = dataset_inputs_B_run.rename(columns={'full_text_x':'full_text'})

In [13]:
# B_run_temp = pd.read_csv('priority/b_run/dataset_all.csv')
# dataset_inputs_B_run = pd.merge(cv19_b_run_info_labeled,B_run_temp,on=['id'])
dataset_inputs_B_run = cv19_b_run_info_labeled

In [14]:
len(dataset_inputs_B_run)

4505

In [15]:
dataset_inputs_B_run.full_text

0       Texas-78/1 4:36 Daily-Total 438293, Up 10064, ...
1       Texas counties are requesting refrigerated tru...
2       Latest health update: link in profile. @ Houst...
3       HTV Houston Television - COVID-19 Talk with Lo...
4       All Masks, Disinfectant Wipes/Sprays &amp; San...
                              ...                        
4500    The commissioner for St. Johns County just sou...
4501    Updated: Emergency vehicles in Duval County on...
4502    Jacksonville #Jaguars ativou o RB Ryquell Arms...
4503    Prayers for Lt.Cunningham's family &amp; his c...
4504    Alexander Bernardo, 22yo Dietary Aide, Park Ri...
Name: full_text, Length: 4505, dtype: object

In [16]:
sentences_A_run = dataset_inputs_A_run.full_text.values
# labels_A_run = dataset_inputs_A_run.target.values
sentences_B_run = dataset_inputs_B_run.full_text.values
# labels_B_run = dataset_inputs_B_run.target.values

In [17]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


Loading BERT tokenizer...


In [18]:
for idx in range(len(sentences_A_run)):
  sentences_A_run[idx] = sentences_A_run[idx]  + ' '+ dataset_inputs_A_run.hashtags.values[idx] + ' ' + str(dataset_inputs_A_run.favorite_count.values[idx])

# for idx in range(len(sentences_B_run)):
#   sentences_B_run[idx] = sentences_B_run[idx]  + ' '+ dataset_inputs_B_run.hashtags.values[idx] + ' ' + str(dataset_inputs_B_run.favorite_count.values[idx])
for idx in range(len(sentences_B_run)):
  sentences_B_run[idx] = sentences_B_run[idx]  +' ' + str(dataset_inputs_B_run.favorite_count.values[idx])


In [19]:
def Encode_sents(sentences):
    input_ids = []
    # For every sentence...
    for sent in sentences:
        # `encode` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        encoded_sent = tokenizer.encode(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'


                            #max_length = 128,          # Truncate all sentences.
                            #return_tensors = 'pt',     # Return pytorch tensors.
                       )

        # Add the encoded sentence to the list.
        # If the length of token is over the bert limitation, cut off the back
        if(len(encoded_sent)>144):
            diff = int((len(encoded_sent) - 145)/2)
            encoded_sent = encoded_sent[diff:144+diff]
        input_ids.append(encoded_sent)
    return input_ids



In [20]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids_A_run = Encode_sents(sentences_A_run)
input_ids_B_run = Encode_sents(sentences_B_run)



## Padding


In [21]:

# We'll borrow the `pad_sequences` utility function to do this.
from keras.preprocessing.sequence import pad_sequences

# Set the maximum sequence length.

MAX_LEN = 144

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

# Pad our input tokens with value 0.
# "post" indicates that we want to pad and truncate at the end of the sequence,
# as opposed to the beginning.
input_ids_A_run = pad_sequences(input_ids_A_run, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

input_ids_B_run = pad_sequences(input_ids_B_run, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")
print('\nDone.')


Padding/truncating all sentences to 144 values...

Padding token: "[PAD]", ID: 0

Done.


Using TensorFlow backend.


## Attension mask

In [22]:
# Create attention masks
def build_attension_mask(input_ids):
    attention_masks = []
    # For each sentence...
    for sent in input_ids:

        # Create the attention mask.
        #   - If a token ID is 0, then it's padding, set the mask to 0.
        #   - If a token ID is > 0, then it's a real token, set the mask to 1.
        att_mask = [int(token_id > 0) for token_id in sent]

        # Store the attention mask for this sentence.
        attention_masks.append(att_mask)
    return attention_masks


In [23]:
attention_masks_A_run = build_attension_mask(input_ids_A_run)
attention_masks_B_run = build_attension_mask(input_ids_B_run)


# Classication one category by on category

All training split dataset first. Then use BERT to evaluate and store the result into a file.

In [24]:
def BERT_cross_vali_split(input_ids, labels, attention_masks):
    # Use train_test_split to split our data into train and validation and test sets for
    # training
    from sklearn.model_selection import train_test_split

    # Use 80% for training and 20% for validation and  5% for testing.
    temp_inputs, test_inputs,  temp_labels, test_labels = train_test_split(input_ids, labels, test_size=0.05,random_state=2020)
    train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(temp_inputs, temp_labels, test_size=0.2,random_state=2020)

    # Do the same for the masks.
    temp_masks, test_masks, temp_labels, _ = train_test_split(attention_masks, labels, test_size=0.05,random_state=2020)

    train_masks, validation_masks, _, _ = train_test_split(temp_masks, temp_labels, test_size=0.2,random_state=2020)
    return train_inputs,  train_labels, train_masks, validation_inputs, validation_labels, validation_masks, test_inputs, test_labels, test_masks 

In [25]:
# A dict to store all metrics fro each category, Use name of category as the key to search
all_metrics_all_category = {}


In [26]:
def bert_test_one_dataset(model_pth,test_batch,batch_size,category_number):
    # Build new BERT model
    model  = BertForSequenceClassification.from_pretrained(
      "bert-base-uncased",  # Use the 12-layer BERT model, with an uncased vocab.
      num_labels=category_number,  # The number of output labels--2 for binary classification.
      # You can increase this for multi-class tasks.
      output_attentions=False,  # Whether the model returns attentions weights.
      output_hidden_states=False,  # Whether the model returns all hidden-states.
    )
    # send model to GPU

    model.load_state_dict(torch.load(model_pth))


    test_inputs = torch.tensor(test_batch[0]).to(torch.int64)
    test_labels = torch.tensor(test_batch[1]).to(torch.int64)
    test_masks = torch.tensor(test_batch[2]).to(torch.int64)
    test_data = TensorDataset(test_inputs, test_masks, test_labels)
    sampler = RandomSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=sampler, batch_size=batch_size)
    
    # ========================================
    #               Testing
    # ========================================
    print()
    print('Testing...')
    # record the start time
    t0 = time.time()

    # Identify GPU to use
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print('There are %d GPU(s) available.' % torch.cuda.device_count())
        print('We will use the GPU:', torch.cuda.get_device_name(0))
    else:
        print('No GPU available, using the CPU instead.')
        device = torch.device("cpu")
    # put model in evaluation model
    model.eval()

    model.to(device)
    # Build list to store metrics of each epoches
    test_acc_all = []
    test_precision_all = []
    test_recall_all = []
    test_f1_all = []
    test_accuracy = 0
    nb_test_steps, nb_test_examples = 0, 0

    # Build the dict for each metrics and store metrics for each category
    test_precision, test_recall, test_f1 = {}, {}, {}
    for i in range(category_number):
        test_f1[i], test_precision[i], test_recall[i] = 0, 0, 0

    # For each batch of training data...
    for batch in test_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)

        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():

          outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
          # Get the "logits" output by the model. The "logits" are the output
          # values prior to applying an activation function like the softmax.
          logits = outputs[0]

          # Move logits and labels to CPU
          logits = logits.detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()

          # Calculate the accuracy for this batch of test sentences.
          tmp_test_accuracy = Scoring_nn.flat_accuracy(None, logits, label_ids)

          tmp_test_precision = {}
          tmp_test_recall = {}
          tmp_test_f1 = {}
          for i in range(category_number):
            tmp_test_precision[i] = 0
            tmp_test_recall[i] = 0
            tmp_test_f1[i] = 0

          for k in range(category_number):
            tmp_test_precision[k], tmp_test_recall[k], tmp_test_f1[k] = Scoring_nn.flat_metrics(None, logits, label_ids, k)

          # Accumulate the total accuracy.
          test_accuracy += tmp_test_accuracy

          for k in range(category_number):
            test_precision[k] += tmp_test_precision[k]
            test_recall[k] += tmp_test_recall[k]
            test_f1[k] += tmp_test_f1[k]

          # Track the number of batches
          nb_test_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(test_accuracy / nb_test_steps))

    # Calculate the average value for each metrics
    precision_avg, recall_avg, f1_avg = 0, 0, 0
    for k in range(category_number):
        print("Category: %d" % (k))
        test_precision[k] = test_precision[k] / nb_test_steps
        precision_avg += test_precision[k]
        test_recall[k] = test_recall[k] / nb_test_steps
        recall_avg += test_recall[k]
        test_f1[k] = test_f1[k] / nb_test_steps
        f1_avg += test_f1[k]
        print("  Precision: {0:.4f}".format(test_precision[k]))
        print("  Recall: {0:.4f}".format(test_recall[k]))
        print("  F1: {0:.4f}".format(test_f1[k]))
    # Report other metrics
    print("The average precision is: {0:.4f}".format(precision_avg / category_number))
    print("The average recall is: {0:.4f}".format(recall_avg / category_number))
    print("The average f1 is: {0:.4f}".format(f1_avg / category_number))
    print("  Testing took: {:}".format(Scoring_nn.format_time(time.time() - t0)))
    
    del model, b_input_ids, b_input_mask, b_labels, outputs, logits, label_ids
    torch.cuda.empty_cache()

### For 'GoodsServices'

#### Split

#### Incremental recitfied training

In [27]:
def cross_event_validate(section_category, a_batch, b_batch,
                        ):
    import json
    
    # training
    section_category = section_category
    a_labels = a_batch[1][section_category]
    b_labels = b_batch[1][section_category]

    train_inputs_A_run,  train_labels_A_run, train_masks_A_run, validation_inputs_A_run, validation_labels_A_run, validation_masks_A_run,test_inputs_A_run, test_labels_A_run, test_masks_A_run = BERT_cross_vali_split(a_batch[0], a_labels, a_batch[2])

    train_inputs_B_run,  train_labels_B_run, train_masks_B_run, validation_inputs_B_run, validation_labels_B_run, validation_masks_B_run,test_inputs_B_run, test_labels_B_run, test_masks_B_run = BERT_cross_vali_split(b_batch[0], b_labels, b_batch[2])
    test_batch_A_run = [test_inputs_A_run,test_labels_A_run,test_masks_A_run]
    test_batch_B_run = [test_inputs_B_run,test_labels_B_run,test_masks_B_run]

    batch_size = 32
    lr = 3e-5
    category_number = 2
    input_ids_A_run,  attention_masks_A_run = a_batch[0],a_batch[2]
    input_ids_B_run, attention_masks_B_run = b_batch[0], b_batch[2]
    
    print('Train on dataset-a and test on dataset-b')
    print('Train on dataset-a and test on dataset-b')
    # Train on dataset-a and test on dataset-b
    Bert = Bertnn_rectified_info(train_inputs_A_run,  train_labels_A_run, train_masks_A_run,
                                 validation_inputs_A_run, validation_labels_A_run, validation_masks_A_run, test_batch_B_run, batch_size=batch_size,lr=lr)
    
    all_metrics_one, all_tesst_metrics = Bert.searchUpsample(30)
    all_metrics_one['test'] = all_tesst_metrics
    all_metrics_all_category[section_category] = all_metrics_one
    if not os.path.exists('BertSearchResult'):
      os.makedirs('BertSearchResult')

    filename = './BertSearchResult/'+'rectified'+'_'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)+ 'train_on_a'
    with open(filename,'w') as file_obj:
      json.dump(all_metrics_one,file_obj)
      print('Successfully save file %s'%(filename))

    del Bert
    torch.cuda.empty_cache() 

    model_pth_old = './Best_val_Bert_rectified_binary.pth'
    model_pth = './Best_info_rectified_best_'+str(section_category)+'_train_on_a_'+'.pth'
    os.rename(model_pth_old,model_pth)    
    test_batch = [input_ids_A_run, a_labels, attention_masks_A_run]
    bert_test_one_dataset(model_pth,test_batch,batch_size,category_number)
    
    
    print('Train on dataset-b and test on dataset-a')
    print('Train on dataset-b and test on dataset-a')
    # Train on dataset-b and test on dataset-a
    Bert = Bertnn_rectified_info(train_inputs_B_run,  train_labels_B_run, train_masks_B_run, validation_inputs_B_run, validation_labels_B_run, validation_masks_B_run, test_batch_A_run, batch_size=batch_size,lr=lr)
    all_metrics_one, all_tesst_metrics = Bert.searchUpsample(30)
    all_metrics_one['test'] = all_tesst_metrics
    all_metrics_all_category[section_category] = all_metrics_one
    if not os.path.exists('BertSearchResult'):
      os.makedirs('BertSearchResult')

    filename = './BertSearchResult/'+'rectified'+'_'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category) + 'train_on_b'
    with open(filename,'w') as file_obj:
      json.dump(all_metrics_one,file_obj)
      print('Successfully save file %s'%(filename))

    del Bert
    torch.cuda.empty_cache() 

    model_pth_old = './Best_val_Bert_rectified_binary.pth'
    model_pth = './Best_info_rectified_best_'+str(section_category)+'_train_on_b_'+'.pth'
    os.rename(model_pth_old,model_pth)    

    test_batch = [input_ids_B_run, b_labels, attention_masks_B_run]
    bert_test_one_dataset(model_pth,test_batch,batch_size,category_number)
    

In [1]:
import json
batch_size = 16
lr = 3e-5

category_number = 2
# Train on dataset-a and test on dataset-b
Bert = Bertnn_rectified_info(train_inputs_A_run,  train_labels_A_run, train_masks_A_run, validation_inputs_A_run, validation_labels_A_run, validation_masks_A_run, test_batch_B_run, batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(30)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+'rectified'+'_'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)+ 'train_on_a'
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

model_pth_old = './Best_val_Bert_rectified_binary.pth'
model_pth = './Best_info_rectified_best_'+str(section_category)+'_train_on_a_'+'.pth'
os.rename(model_pth_old,model_pth)    

NameError: name 'Bertnn_rectified_info' is not defined

In [ ]:
all_metrics_one['test']

In [ ]:
model_pth = './Best_info_rectified_best_'+str(section_category)+'_train_on_a_'+'.pth'
test_batch = [input_ids_A_run, a_labels, attention_masks_A_run]
bert_test_one_dataset(model_pth,test_batch,batch_size,category_number)

In [ ]:

# Train on dataset-b and test on dataset-a
Bert = Bertnn_rectified_info(train_inputs_B_run,  train_labels_B_run, train_masks_B_run, validation_inputs_B_run, validation_labels_B_run, validation_masks_B_run, test_batch_A_run, batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(30)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+'rectified'+'_'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category) + 'train_on_b'
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

model_pth_old = './Best_val_Bert_rectified_binary.pth'
model_pth = './Best_info_rectified_best_'+str(section_category)+'_train_on_b_'+'.pth'
os.rename(model_pth_old,model_pth)    

test_batch = [input_ids_B_run, b_labels, attention_masks_B_run]
bert_test_one_dataset(model_pth,test_batch,batch_size,category_number)

### For 'InformationWanted'

#### Cross event validation

In [ ]:
section_category = 1
# train_batch_a    = train_inputs_A_run,  train_labels_A_run, train_masks_A_run 
# val_batch_a    = validation_inputs_A_run, validation_labels_A_run, validation_masks_A_run
a_batch = input_ids_A_run, labels_single_a, attention_masks_A_run 

# train_batch_b = train_inputs_B_run,  train_labels_B_run, train_masks_B_run 
# val_batch_b =    validation_inputs_B_run, validation_labels_B_run, validation_masks_B_run 
b_batch =     input_ids_B_run, labels_single_b, attention_masks_B_run 
    
cross_event_validate(section_category, a_batch, b_batch,
                        )


#### Normal training

In [ ]:
import json
batch_size = 16
lr = 2e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

In [ ]:
all_metrics_one

#### Incremental recitfied training

In [ ]:
import json
batch_size = 32
lr = 3e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_rectified_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+'rectified'+'_'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

In [ ]:
all_metrics_one

### For 'Volunteer'

#### Split

In [ ]:
section_category = 2
# train_batch_a    = train_inputs_A_run,  train_labels_A_run, train_masks_A_run 
# val_batch_a    = validation_inputs_A_run, validation_labels_A_run, validation_masks_A_run
a_batch = input_ids_A_run, labels_single_a, attention_masks_A_run 

# train_batch_b = train_inputs_B_run,  train_labels_B_run, train_masks_B_run 
# val_batch_b =    validation_inputs_B_run, validation_labels_B_run, validation_masks_B_run 
b_batch =     input_ids_B_run, labels_single_b, attention_masks_B_run 
    
cross_event_validate(section_category, a_batch, b_batch,
                        )


#### Normal training

In [ ]:
import json
batch_size = 16
lr = 2e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

In [ ]:
all_metrics_one

#### Incremental recitfied training

In [ ]:
import json
batch_size = 32
lr = 3e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_rectified_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+'rectified'+'_'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

In [2]:
all_metrics_one['test']

NameError: name 'all_metrics_one' is not defined

### For 'EmergingThreats'

#### Split

In [ ]:
section_category = 3
# train_batch_a    = train_inputs_A_run,  train_labels_A_run, train_masks_A_run 
# val_batch_a    = validation_inputs_A_run, validation_labels_A_run, validation_masks_A_run
a_batch = input_ids_A_run, labels_single_a, attention_masks_A_run 

# train_batch_b = train_inputs_B_run,  train_labels_B_run, train_masks_B_run 
# val_batch_b =    validation_inputs_B_run, validation_labels_B_run, validation_masks_B_run 
b_batch =     input_ids_B_run, labels_single_b, attention_masks_B_run 
    
cross_event_validate(section_category, a_batch, b_batch,
                        )


#### Normal training

#### Incremental recitfied training

### For 'NewSubEvent'

#### Split

In [3]:
section_category = 4
# train_batch_a    = train_inputs_A_run,  train_labels_A_run, train_masks_A_run 
# val_batch_a    = validation_inputs_A_run, validation_labels_A_run, validation_masks_A_run
a_batch = input_ids_A_run, labels_single_a, attention_masks_A_run 

# train_batch_b = train_inputs_B_run,  train_labels_B_run, train_masks_B_run 
# val_batch_b =    validation_inputs_B_run, validation_labels_B_run, validation_masks_B_run 
b_batch =     input_ids_B_run, labels_single_b, attention_masks_B_run 
    
cross_event_validate(section_category, a_batch, b_batch,
                        )


NameError: name 'input_ids_A_run' is not defined

#### Normal training

In [ ]:
import json
batch_size = 16
lr = 2e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

In [ ]:
all_metrics_one

#### Incremental recitfied training

In [ ]:
import json
batch_size = 32
lr = 3e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_rectified_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+'rectified'+'_'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

In [ ]:
all_metrics_one['test']

### For 'ServiceAvailable'

#### Split

In [28]:
section_category = 5
# train_batch_a    = train_inputs_A_run,  train_labels_A_run, train_masks_A_run 
# val_batch_a    = validation_inputs_A_run, validation_labels_A_run, validation_masks_A_run
a_batch = input_ids_A_run, labels_single_a, attention_masks_A_run 

# train_batch_b = train_inputs_B_run,  train_labels_B_run, train_masks_B_run 
# val_batch_b =    validation_inputs_B_run, validation_labels_B_run, validation_masks_B_run 
b_batch =     input_ids_B_run, labels_single_b, attention_masks_B_run 
    
cross_event_validate(section_category, a_batch, b_batch,
                        )


Train on dataset-a and test on dataset-b
Train on dataset-a and test on dataset-b
There are 1 GPU(s) available.
We will use the GPU: TITAN RTX


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Current increase 0 times
Current is processing category 1
Before up-sample, there are 5762 samples
There are 278 sampled in this category.
After up-sample, there are 5762 samples

Currrent is processing on categorie GoodsServices

Before up-sample, the ratio of current category and all samples
0
0.0

After up-sample, the ratio of current category and all samples
5484
0.9517528635890316

Currrent is processing on categorie InformationWanted

Before up-sample, the ratio of current category and all samples
0
0.0

After up-sample, the ratio of current category and all samples
278
0.048247136410968415



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

There are 1 GPU(s) available.
We will use the GPU: TITAN RTX

======== Epoch 1 / 4 ========
Training...
  Batch    40  of    361.    Elapsed: 0:00:11.
  Batch    80  of    361.    Elapsed: 0:00:23.
  Batch   120  of    361.    Elapsed: 0:00:34.
  Batch   160  of    361.    Elapsed: 0:00:45.
  Batch   200  of    361.    Elapsed: 0:00:57.
  Batch   240  of    361.    Elapsed: 0:01:08.
  Batch   280  of    361.    Elapsed: 0:01:20.
  Batch   320  of    361.    Elapsed: 0:01:31.
  Batch   360  of    361.    Elapsed: 0:01:42.

  Average training loss: 1.32
  Accuracy: 0.95
Category: 0
  Precision: 0.96
  Recall: 0.99
  F1: 0.98
Category: 1
  Precision: 0.10
  Recall: 0.08
  F1: 0.08
The average precision is: 0.5278
The average recall is: 0.5390
The average f1 is: 0.5298
  Training epcoh took: 0:01:42

Running Validation...
  Accuracy: 0.96
Category: 0
  Precision: 0.9673
  Recall: 0.9949
  F1: 0.9803
Category: 1
  Precision: 0.1154
  Recall: 0.0934
  F1: 0.1007
The average precision is: 0.5

...

There are 1 GPU(s) available.
We will use the GPU: TITAN RTX

======== Epoch 1 / 4 ========
Training...
  Batch    40  of    290.    Elapsed: 0:00:13.
  Batch    80  of    290.    Elapsed: 0:00:27.
  Batch   120  of    290.    Elapsed: 0:00:40.
  Batch   160  of    290.    Elapsed: 0:00:54.
  Batch   200  of    290.    Elapsed: 0:01:07.
  Batch   240  of    290.    Elapsed: 0:01:20.
  Batch   280  of    290.    Elapsed: 0:01:34.

  Average training loss: 0.25
  Accuracy: 0.94
Category: 0
  Precision: 0.96
  Recall: 0.96
  F1: 0.96
Category: 1
  Precision: 0.87
  Recall: 0.88
  F1: 0.86
The average precision is: 0.9134
The average recall is: 0.9193
The average f1 is: 0.9081
  Training epcoh took: 0:01:37

Running Validation...
  Accuracy: 0.98
Category: 0
  Precision: 0.9790
  Recall: 0.9965
  F1: 0.9873
Category: 1
  Precision: 0.2222
  Recall: 0.2037
  F1: 0.2099
The average precision is: 0.6006
The average recall is: 0.6001
The average f1 is: 0.5986
  Validation took: 0:00:03

======

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

There are 1 GPU(s) available.
We will use the GPU: TITAN RTX

======== Epoch 1 / 4 ========
Training...
  Batch    40  of    366.    Elapsed: 0:00:14.
  Batch    80  of    366.    Elapsed: 0:00:28.
  Batch   120  of    366.    Elapsed: 0:00:42.
  Batch   160  of    366.    Elapsed: 0:00:57.
  Batch   200  of    366.    Elapsed: 0:01:11.
  Batch   240  of    366.    Elapsed: 0:01:25.
  Batch   280  of    366.    Elapsed: 0:01:39.
  Batch   320  of    366.    Elapsed: 0:01:58.
  Batch   360  of    366.    Elapsed: 0:02:24.

  Average training loss: 0.11
  Accuracy: 0.94
Category: 0
  Precision: 0.96
  Recall: 0.94
  F1: 0.94
Category: 1
  Precision: 0.92
  Recall: 0.95
  F1: 0.92
The average precision is: 0.9403
The average recall is: 0.9420
The average f1 is: 0.9333
  Training epcoh took: 0:02:26

Running Validation...
  Accuracy: 0.97
Category: 0
  Precision: 0.9750
  Recall: 0.9913
  F1: 0.9826
Category: 1
  Precision: 0.1790
  Recall: 0.1605
  F1: 0.1636
The average precision is: 0.5

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at


Testing...
There are 1 GPU(s) available.
We will use the GPU: TITAN RTX
  Accuracy: 0.99
Category: 0
  Precision: 0.9952
  Recall: 0.9945
  F1: 0.9947
Category: 1
  Precision: 0.6327
  Recall: 0.6424
  F1: 0.6230
The average precision is: 0.8140
The average recall is: 0.8185
The average f1 is: 0.8089
  Testing took: 0:00:15


#### Normal training

In [ ]:
import json
batch_size = 16
lr = 2e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

In [ ]:
all_metrics_one

#### Incremental recitfied training

In [4]:
import json
batch_size = 32
lr = 3e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_rectified_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+'rectified'+'_'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

NameError: name 'test_inputs' is not defined

In [ ]:
all_metrics_one

### For 'Advice'

#### Split

In [28]:
section_category = 6
# train_batch_a    = train_inputs_A_run,  train_labels_A_run, train_masks_A_run 
# val_batch_a    = validation_inputs_A_run, validation_labels_A_run, validation_masks_A_run
a_batch = input_ids_A_run, labels_single_a, attention_masks_A_run 

# train_batch_b = train_inputs_B_run,  train_labels_B_run, train_masks_B_run 
# val_batch_b =    validation_inputs_B_run, validation_labels_B_run, validation_masks_B_run 
b_batch =     input_ids_B_run, labels_single_b, attention_masks_B_run 
    
cross_event_validate(section_category, a_batch, b_batch,
                        )


Train on dataset-a and test on dataset-b
Train on dataset-a and test on dataset-b
There are 1 GPU(s) available.
We will use the GPU: TITAN RTX


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Current increase 0 times
Current is processing category 1
Before up-sample, there are 5762 samples
There are 443 sampled in this category.
After up-sample, there are 5762 samples

Currrent is processing on categorie GoodsServices

Before up-sample, the ratio of current category and all samples
0
0.0

After up-sample, the ratio of current category and all samples
5319
0.923116973273169

Currrent is processing on categorie InformationWanted

Before up-sample, the ratio of current category and all samples
0
0.0

After up-sample, the ratio of current category and all samples
443
0.07688302672683096



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

There are 1 GPU(s) available.
We will use the GPU: TITAN RTX

======== Epoch 1 / 4 ========
Training...
  Batch    40  of    361.    Elapsed: 0:00:12.
  Batch    80  of    361.    Elapsed: 0:00:24.
  Batch   120  of    361.    Elapsed: 0:00:36.
  Batch   160  of    361.    Elapsed: 0:00:48.
  Batch   200  of    361.    Elapsed: 0:01:00.
  Batch   240  of    361.    Elapsed: 0:01:11.
  Batch   280  of    361.    Elapsed: 0:01:23.
  Batch   320  of    361.    Elapsed: 0:01:35.
  Batch   360  of    361.    Elapsed: 0:01:47.

  Average training loss: 1.86
  Accuracy: 0.92
Category: 0
  Precision: 0.92
  Recall: 1.00
  F1: 0.96
Category: 1
  Precision: 0.00
  Recall: 0.00
  F1: 0.00
The average precision is: 0.4619
The average recall is: 0.5008
The average f1 is: 0.4794
  Training epcoh took: 0:01:47

Running Validation...
  Accuracy: 0.92
Category: 0
  Precision: 0.9224
  Recall: 1.0000
  F1: 0.9583
Category: 1
  Precision: 0.0000
  Recall: 0.0000
  F1: 0.0000
The average precision is: 0.4

...

There are 1 GPU(s) available.
We will use the GPU: TITAN RTX

======== Epoch 1 / 4 ========
Training...
  Batch    40  of    307.    Elapsed: 0:00:14.
  Batch    80  of    307.    Elapsed: 0:00:27.
  Batch   120  of    307.    Elapsed: 0:00:41.
  Batch   160  of    307.    Elapsed: 0:00:54.
  Batch   200  of    307.    Elapsed: 0:01:34.
  Batch   240  of    307.    Elapsed: 0:01:54.
  Batch   280  of    307.    Elapsed: 0:02:08.

  Average training loss: 0.22
  Accuracy: 0.93
Category: 0
  Precision: 0.95
  Recall: 0.95
  F1: 0.95
Category: 1
  Precision: 0.90
  Recall: 0.90
  F1: 0.89
The average precision is: 0.9242
The average recall is: 0.9258
The average f1 is: 0.9155
  Training epcoh took: 0:02:17

Running Validation...
  Accuracy: 0.95
Category: 0
  Precision: 0.9646
  Recall: 0.9856
  F1: 0.9739
Category: 1
  Precision: 0.1451
  Recall: 0.1235
  F1: 0.1204
The average precision is: 0.5548
The average recall is: 0.5546
The average f1 is: 0.5471
  Validation took: 0:00:03

======

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

There are 1 GPU(s) available.
We will use the GPU: TITAN RTX

======== Epoch 1 / 4 ========
Training...
  Batch    40  of    399.    Elapsed: 0:00:14.
  Batch    80  of    399.    Elapsed: 0:00:29.
  Batch   120  of    399.    Elapsed: 0:00:43.
  Batch   160  of    399.    Elapsed: 0:00:58.
  Batch   200  of    399.    Elapsed: 0:01:12.
  Batch   240  of    399.    Elapsed: 0:01:26.
  Batch   280  of    399.    Elapsed: 0:01:41.
  Batch   320  of    399.    Elapsed: 0:01:55.
  Batch   360  of    399.    Elapsed: 0:02:09.

  Average training loss: 0.19
  Accuracy: 0.91
Category: 0
  Precision: 0.91
  Recall: 0.88
  F1: 0.89
Category: 1
  Precision: 0.90
  Recall: 0.94
  F1: 0.91
The average precision is: 0.9067
The average recall is: 0.9102
The average f1 is: 0.8984
  Training epcoh took: 0:02:23

Running Validation...
  Accuracy: 0.95
Category: 0
  Precision: 0.9690
  Recall: 0.9807
  F1: 0.9740
Category: 1
  Precision: 0.1852
  Recall: 0.1883
  F1: 0.1753
The average precision is: 0.5

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at


Testing...
There are 1 GPU(s) available.
We will use the GPU: TITAN RTX
  Accuracy: 0.99
Category: 0
  Precision: 0.9913
  Recall: 0.9949
  F1: 0.9929
Category: 1
  Precision: 0.6048
  Recall: 0.5872
  F1: 0.5792
The average precision is: 0.7980
The average recall is: 0.7911
The average f1 is: 0.7861
  Testing took: 0:00:15


#### Normal training

In [ ]:
import json
batch_size = 16
lr = 2e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

In [ ]:
all_metrics_one

#### Incremental recitfied training

In [ ]:
import json
batch_size = 32
lr = 3e-5
test_batch = [test_inputs,test_labels,test_masks]
Bert = Bertnn_rectified_info(train_inputs,train_labels,train_masks, validation_inputs,validation_labels,validation_masks,test_batch,batch_size=batch_size,lr=lr)
all_metrics_one, all_tesst_metrics = Bert.searchUpsample(100)
all_metrics_one['test'] = all_tesst_metrics
all_metrics_all_category[section_category] = all_metrics_one
if not os.path.exists('BertSearchResult'):
  os.makedirs('BertSearchResult')

filename = './BertSearchResult/'+'rectified'+'_'+str(section_category)+'_'+str(batch_size) + '_' + str(lr) + '_'  + str(section_category)
with open(filename,'w') as file_obj:
  json.dump(all_metrics_one,file_obj)
  print('Successfully save file %s'%(filename))

del Bert
torch.cuda.empty_cache() 

In [ ]:
all_metrics_one